In [18]:
!pip install beir

In [19]:
! pip install datasets transformers

# Partie 1

In [20]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

In [21]:
dataset = "dbpedia"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/4635922 [00:00<?, ?it/s]

In [22]:
from datasets import load_dataset, load_metric

datasets = load_dataset("squad_v2")

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
# import itertools
# corpus = dict(itertools.islice(corpus.items(), 15000))

In [24]:
def context_50words(corpus):
    context_list = []
    
    for elt in corpus.keys():
        #corpus_in_tokens = corpus[elt]['text'].split(' ')
        corpus_in_words = [word.lower() for word in corpus_in_tokens if word != '' and all(c.isalpha() for c in word)]
    
        if corpus_in_words in context_list:
            continue
    
        if len(corpus_in_words) >= 50:
            context_list.append(corpus_in_words)
    
    return context_list

In [25]:
class PreProcessCorpus:
    
    def __init__(self):
        question = []
        context = []
        answer = None
    
    def get_attr_qrel(self, qrel_key, queries, context_list):
        doc = qrels[qrel_key].keys()
        if doc['text'] in context_list:
            self.question = queries[qrel_key]
            title = self.word_to_list(doc['title'])
            self.answer = Answer(context, title)
        
    def word_to_list(self, text):
        tokens = " ".join(text)
        words = [word.lower() for word in token.split() if all(c.isalpha() for c in word)]
        
    def context_list_builder(self, corpus):
        new_dataset = []

        for elt in corpus.keys():
            corpus_in_tokens = " ".join(corpus[elt]['text'])
            corpus_in_words = [word.lower() for word in corpus_in_tokens.split() if all(c.isalpha() for c in word)]

            if corpus_in_words in new_dataset:
                continue

            if len(corpus_in_words) >= 50:
                new_dataset.append(corpus_in_words)
        return new_dataset

In [26]:
class Corpus:
  def context_50words(self, corpus):
    context_list = []
    
    for elt in corpus.keys():
        corpus_in_tokens = corpus[elt]['text'].split(' ')
        corpus_in_words = [word.lower() for word in corpus_in_tokens if word != '' and all(c.isalpha() for c in word)]
    
        if corpus_in_words in context_list:
            continue
    
        if len(corpus_in_words) >= 50:
            context_list.append(corpus_in_words)
    
    return context_list

In [27]:
print(queries)

{'INEX_LD-2009022': 'Szechwan dish food cuisine', 'INEX_LD-2009039': 'roman architecture', 'INEX_LD-2009053': 'finland car industry manufacturer saab sisu', 'INEX_LD-2009061': 'france second world war normandy', 'INEX_LD-2009062': 'social network group selection', 'INEX_LD-2009063': 'D-Day normandy invasion', 'INEX_LD-2009074': 'web ranking scoring algorithm', 'INEX_LD-2009115': 'virtual museums', 'INEX_LD-2010004': 'Indian food', 'INEX_LD-2010014': 'composer museum', 'INEX_LD-2010019': 'gallo roman architecture in paris', 'INEX_LD-2010020': 'electricity source in France', 'INEX_LD-2010037': 'social network API', 'INEX_LD-2010043': 'List of films from the surrealist category', 'INEX_LD-2010057': 'Einstein Relativity theory', 'INEX_LD-2010069': 'summer flowers', 'INEX_LD-2010100': 'house concrete wood', 'INEX_LD-2010106': 'organic food advantages disadvantages', 'INEX_LD-20120111': 'vietnam war movie', 'INEX_LD-20120121': 'vietnam food recipes', 'INEX_LD-20120122': 'vietnamese food blog

In [28]:
class Entry:
  def __init__(self, context, title, question, answer):
    self.question = question
    self.answer = answer
    self.context = context
    self.title = title

  def entry_to_list(self):
    return [context, title, question, answer]

  def get_question(self):
    return question
  def get_answer(self):
    return answer
  def get_context(self):
    return context
  def get_title(self):
    return title

In [29]:
def extract_title(query_title):
  res = query_title[9:len(query_title) - 1]
  return res

In [59]:
all_good_entries = []

all_questions = []

for i in qrels:
  # Check if query is a question (because we want a question-answer thing)
  if "QALD2" in i:
    # get question
    question = queries[i]
    all_questions.append(question)
    # for every entry that correspond to this question
    for j in qrels[i]:
      # get title and context
      answer = corpus[j]['title']
      context = corpus[j]['text']
      # check if it's long enough for us
      context_tokens = context.split(' ')
      context_words = [word.lower() for word in context_tokens if word != '' and all(c.isalpha() for c in word)]
      if len(context_words) >= 50:
        # create a new entry
        title = extract_title(j)
        entry = [context, title, question, answer]
        all_good_entries.append(entry)

In [31]:
# format: context, title, question, answer

all_good_entries

[['El Asher City or Tenth of Ramadan City (Arabic: مدينة العاشر من رمضان\u200e) is a city located in Sharqia governorate in Egypt and follows the Urban Communities Authority. It is one of the cities of the first generation, and one of the largest new industrial cities, located close to the city of Cairo. It was created by Presidential Decree No.',
  '10th_of_Ramadan_City',
  'Which German cities have more than 250000 inhabitants?',
  '10th of Ramadan City'],
 ["Abdul Hafeez Kardar (Urdu: عبد الحفیظ کاردار  \u200e) About this sound pronunciation  or Abdul Kardar (17 January 1925 – 21 April 1996) was an international cricketer, who is one of the only three players  to have played Test cricket for both India and Pakistan, the other two being Amir Elahi and Gul Mohammad. He became the first captain of the Pakistan cricket team and is widely regarded as a father figure of Pakistan's cricket.",
  'Abdul_Hafeez_Kardar',
  'Which German cities have more than 250000 inhabitants?',
  'Abdul Hafe

In [32]:
import itertools
test = dict(itertools.islice(corpus.items(), 3))

In [33]:
len(all_good_entries)

7750

In [60]:
count = 0

dico = {}
for i in range(len(datasets['validation'])):
  context = datasets['validation'][i]['context']
  if context not in dico:
    new_dico = {}
    new_dico['title'] = datasets['validation'][i]['title']
    new_dico['question'] = datasets['validation'][i]['question']
    all_questions.append(datasets['validation'][i]['question'])
    new_dico['answers'] = datasets['validation'][i]['answers']
    dico[context] = new_dico

In [49]:
datasets['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [50]:
for i in range(len(all_good_entries)):
  if all_good_entries[i][0] not in dico:
    new_dico = {}
    entry = all_good_entries[i]
    new_dico['title'] = entry[1]
    new_dico['question'] = entry[2]
    new_dico['answers'] = entry[3]
    dico[all_good_entries[i][0]] = new_dico

In [51]:
# key = context, value = dico
#                         => key = title, value = string
#                         => key = question, value = string
#                         => key = answer, value = string

len(dico)

8537

# Partie 2

In [57]:
from sentence_transformers import SentenceTransformer
import sentence_transformers.util

In [58]:
model = SentenceTransformer('msmarco-distilbert-base-v4')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode('London has 9,787,426 inhabitants at the 2011 census')

print("Similarity:", sentence_transformers.util.pytorch_cos_sim(query_embedding, passage_embedding))

Similarity: tensor([[0.5283]])


In [64]:
c = 0

# Testing cosine similarity between a question and its context
for i in dico.keys():
  if c == 5:
    break
  query_embedding = model.encode(dico[i]['question'])
  passage_embedding = model.encode(i)
  print("Similarity:", sentence_transformers.util.pytorch_cos_sim(query_embedding, passage_embedding))
  c += 1

Similarity: tensor([[0.3593]])
Similarity: tensor([[0.1828]])
Similarity: tensor([[0.7451]])
Similarity: tensor([[0.7466]])
Similarity: tensor([[0.4301]])


In [65]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)